# Data Cleaning & Processing
Beschreibt /dokumentiert alle Anpassungen die ihr an den Daten vornehmt. Es ist durchaus
möglich, dass ihr für verschiedene Techniken auch unterschiedliche Anpassungen an den Daten
vornehmt (dann auch jeweils kurz dokumentieren was ihr gemacht habt).

***

In [134]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sb
import re

In [135]:
filename = "./data/survery_data_annonymized_230626.csv"
data = pd.read_csv(filename)

## Erste Erkundung

In [136]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 48 columns):
 #   Column                                                                                                                                                                                                                  Non-Null Count  Dtype 
---  ------                                                                                                                                                                                                                  --------------  ----- 
 0   Timestamp                                                                                                                                                                                                               62 non-null     object
 1   Alter                                                                                                                                                           

### Erkenntnisse
- 48 Spalten mit max. 62 Einträgen
- Datentypen:
    - int64(39), object(9)
- **Sehr lange Spaltenbezeichnungen!**
- **Fehlende Einträge (NaN) bei:**
    - "Was ist dein Lieblingsessen? (nur eines angeben)": 4 Einträge fehlen
    - "Was ist deine Lieblingsband bzw. dein/e Lieblings-Musiker*in? (nur eine/n angeben)": 7 Einträge fehlen

In [137]:
data.describe(include=['int64'])

,Alter,Als wie sportlich würdest du dich einschätzen?,Welche Rolle spielt für dich gesunde Ernährung?,Wie gerne gehst du Essen oder Trinken?,Wie gerne kochst du selbst?,Wie gerne gehst du feiern?,Wie sehr interessierst du dich für Musik?,Wie gerne liest Du?,"Wie sehr interessierst du dich für darstellende Künste wie Schauspiel, Tanz oder Gesang?","Wie sehr interessierst du sich für Filme, Kino, Fernsehsendungen?",...,Strebst du nach dem Studium auch weiterhin eine akademische Laufbahn an?,Wie gern lernst Du in der Gruppe?,"Prokrastinierst du, wenn du lernen müsstest?",Wie häufig machst du Abgaben auf den letzten Drücker?,Wie gerne probierst du neue Lebensmittel aus?,Wie häufig verbringst du Zeit in der Natur (z. B. beim Wandern oder Camping)?,Wie häufig verreist du?,Wie interessiert bist du an handwerklichen Tätigkeiten?,Wie gerne magst du Sprachnachrichten?,Wie wichtig ist es dir viel Geld zu verdienen?
count,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,...,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000,62.000000
mean,25.532258,3.290323,3.548387,3.854839,3.822581,2.822581,4.000000,3.112903,2.629032,3.741935,...,2.596774,3.032258,3.677419,3.403226,3.983871,3.096774,3.000000,3.032258,2.645161,3.370968
std,4.482087,0.981586,0.823376,0.989233,0.983873,1.166811,0.940875,1.229485,1.190590,0.904189,...,1.234207,1.200652,1.098049,1.151758,0.983335,0.935803,1.008163,1.100935,1.242641,0.995628
min,18.000000,1.000000,1.000000,1.000000,2.000000,1.000000,2.000000,1.000000,1.000000,2.000000,...,1.000000,1.000000,1.000000,1.000000,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000
25%,23.000000,3.000000,3.000000,3.000000,3.000000,2.000000,3.000000,2.000000,2.000000,3.000000,...,2.000000,2.000000,3.000000,3.000000,3.250000,2.000000,2.000000,2.000000,2.000000,3.000000
50%,25.000000,3.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.000000,3.000000,4.000000,...,3.000000,3.000000,4.000000,3.000000,4.000000,3.000000,3.000000,3.000000,2.000000,3.000000
75%,28.000000,4.000000,4.000000,5.000000,5.000000,4.000000,5.000000,4.000000,4.000000,4.000000,...,3.000000,4.000000,5.000000,4.000000,5.000000,4.000000,4.000000,4.000000,3.750000,4.000000
max,42.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000


In [138]:
data.describe(include=['object'])

,Timestamp,Geschlecht,"Bitte gib hier ein Pseudonym ein und merke dir dieses (z.B. Kürzel, Zahlencode, etc.). Damit können nach Abschluss des Projektes der eigene ""Datenpunkt"" gefunden werden. Das Pseudonym wird für Andere sichtbar sein.",Welcher Funktion an der HSD fühlst du dich am meisten zugehörig?,Welchem Studiengang fühlst du dich am meisten zugehörig?,Welche Sportarten interessieren dich am meisten (wähle maximal 5)?,Was ist deine absolute Lieblingssportart?,Was ist dein Lieblingsessen? (nur eines angeben),Was ist deine Lieblingsband bzw. dein/e Lieblings-Musiker*in? (nur eine/n angeben)
count,62,62,62,62,62,62,62,58,55
unique,62,2,62,3,4,55,43,38,51
top,2023/05/25 9:05:15 AM GMT+3,männlich,lalala-lololo,Student*in,MMI,Bouldern/Klettern,Fußball,Pizza,Antilopen Gang
freq,1,44,1,59,25,4,6,6,2


### Merkwürdige Einträge
- bei den int64 Einträgen
    - bei 1-5 Fragen (Kategorial, Ordinal?) sollte nichts merkwürdig sein -> ok
    - bei Alter evtl. sowas wie unter 18 und über Rentenalter? Alterspanne von 18-42 -> ok
- bei den object Einträgen
    - Timestamp: keine timestamps die außerhalb des Umfragezeitraum liegen -> ok
    - Geschlecht: ok
    - Pseudonym: keine doppelten oder leeren strings -> ok
    - HSD-Funktion: ok
    - Studiengang: ok
    - Sportarten: ok, keiner hat 0 oder mehr als 5 ausgewählt, aber sollten in einzelne strings aufgeteilt werden, Sportarten die niemand gewählt können raus
    - **Lieblingssportart**: 
        - Nicht-Einträge ('-', 'Weiss nicht', ...) 
        - Mehr-als-1-Einträgen ('E-Sport / CS:GO', 'Kraftsport und Kampfsport' ...)
    - **Lieblingsessen**: 
        - 4 NaN Werte
        - manche sehr detailiert beschrieben (ca. 7)
    - **Lieblingsband**: 
        - 7 NaN Werte
        - Nicht-Einträge ('-', 'Keiner ', 'Alle')

In [139]:
data['Was ist deine absolute Lieblingssportart?'].value_counts()

Fußball                             6
Fußball                             4
Bouldern                            3
Badminton                           3
Tischtennis                         3
Basketball                          2
Volleyball                          2
Klettern                            2
Tauchen                             2
Reiten                              2
/                                   1
Fahrrad fahren                      1
Tanzen                              1
Kraftsport und Kampfsport           1
Hockey                              1
Bogenschießen                       1
Squash                              1
Yoga                                1
Inlineskaten                        1
Joggen/Laufen und Rennrad fahren    1
keine                               1
Ruderboot fahren                    1
Padel-Tennis                        1
Karate                              1
Joggen                              1
E-Sport / CS:GO                     1
Fitness     

In [140]:
data['Was ist dein Lieblingsessen? (nur eines angeben)'].value_counts()

Pizza                                       6
Burger                                      5
Lasagne                                     4
Döner                                       3
Pasta                                       3
Sushi                                       3
Pfannkuchen                                 3
Humus                                       1
Tortelini                                   1
Nudeln                                      1
Yaprak                                      1
Lachs mit Bratkartoffeln und Gurkensalat    1
Ramen                                       1
Vegane Lasagne                              1
Cordon bleu                                 1
Reispfanne mit viel Gemüse                  1
Gemüsepfanne mit Reis                       1
Carbonara                                   1
Ofenkartoffeln mit Gemüse                   1
Pommes 🍟                                    1
Nudeln mit Pilzen in Sahnesauce             1
Hamburger                         

In [141]:
data['Was ist deine Lieblingsband bzw. dein/e Lieblings-Musiker*in? (nur eine/n angeben)'].value_counts()

Antilopen Gang                        2
Radiohead                             2
Mac Miller                            2
Drake                                 2
Kraftklub                             1
Alle                                  1
System of a Down                      1
Sting                                 1
Roger Cicero                          1
Keiner                                1
Avatar                                1
Måneskin                              1
Hans Zimmer                           1
Beethoven                             1
6lack                                 1
King Gizzard and the Lizard Wizard    1
NF                                    1
Deathspell Omega                      1
Red Hot Chili Peppers                 1
Grimes                                1
Die Toten Hosen                       1
Halsey                                1
Ed sheeran                            1
Neffex                                1
Haftbefehl                            1


## Bereinigung
- Spaltenbezeichnungen kürzen
- Textfelder(string): Lieblingssportart/Lieblingsessen/Lieblingsessen
    - NaN-Werte als 'Keine Angabe' definieren
    - Unnötige Leerzeichen entfernen
    - Nicht-Einträge als 'Keine Angabe' vereinheitlichen -> hier könnte NLP angewandt werden zur Klassifizierung von sinnvollen/nicht-sinnvollen Eingaben, aber man müsste alles selbst labeln
    - Abtrennen von Mehr-als-1-Angabe-Einträgen?
- Allgemein: Ähnliche Einträge vereinheitlichen -> NLP
- Ergänzungen
- String-Kategorien als numerische Daten ergänzen

### Spaltenbezeichnungen kürzen
Für eine bessere Lesbarkeit, Übersicht und Verarbeitung werden die Spaltenbezeichnungen angepasst.

In [142]:
to_change = {
    'Bitte gib hier ein Pseudonym ein und merke dir dieses (z.B. Kürzel, Zahlencode, etc.). Damit können nach Abschluss des Projektes der eigene "Datenpunkt" gefunden werden. Das Pseudonym wird für Andere sichtbar sein.':'Pseudonym',
    'Welcher Funktion an der HSD fühlst du dich am meisten zugehörig?': 'Position',
    'Welchem Studiengang fühlst du dich am meisten zugehörig?':'Studiengang',
    'Als wie sportlich würdest du dich einschätzen?':'Sportlichkeit',
    'Welche Sportarten interessieren dich am meisten (wähle maximal 5)?':'Sportarten',
    'Was ist deine absolute Lieblingssportart?':'Fav_Sportart',
    'Welche Rolle spielt für dich gesunde Ernährung? ':'Gesunde_Ernaehrung',
    'Wie gerne gehst du Essen oder Trinken?':'Essen_gehen',
    'Wie gerne kochst du selbst?':'Kochen',
    'Wie gerne gehst du feiern?':'Feiern',
    'Was ist dein Lieblingsessen? (nur eines angeben)':'Fav_Essen',
    'Wie sehr interessierst du dich für Musik?':'Musik',
    'Was ist deine Lieblingsband bzw. dein/e Lieblings-Musiker*in? (nur eine/n angeben)':'Fav_Musik',
    'Wie gerne liest Du?':'Lesen',
    'Wie sehr interessierst du dich für darstellende Künste wie Schauspiel, Tanz oder Gesang?':'Darstellende_Kunst',
    'Wie sehr interessierst du sich für Filme, Kino, Fernsehsendungen?':'Filme',
    'Wie sehr interessierst du dich für bildende Kunst wie Malen, Zeichnen oder Fotografie?':'Bildende_Kunst',
    'Wie gerne besuchst du Museen?':'Museen',
    'Wie sehr interessierst du dich für Videospiele?':'Videospiele',
    'Wie häufig nutzt du soziale Medien?':'Soziale_Medien',
    'Wie groß ist dein Interesse an Technologie und Innovation, wie beispielsweise Programmierung, Robotik und Virtual Reality?':'Technologie',
    'Wie groß ist dein Interesse an Astronomie, Weltraumforschung oder astronomiebezogenen Aktivitäten?':'Astronomie',
    'Wie groß ist dein Interesse an Naturwissenschaften wie Biologie, Chemie und Physik?':'Naturwissenschaften',
    'Wie sehr interessierst du sich für gesellschaftliches Engagement und Freiwilligenarbeit?':'Soziales_Engagement',
    'Wie groß ist dein Interesse an sozialen Themen wie Vielfalt, Gleichberechtigung und soziale Gerechtigkeit?':'Soziale_Gerechtigkeit',
    'Würdest du dich selbst als introvertiert/extrovertiert bezeichnen?':'Introvertiert_Extrovertiert',
    'Hast du häufig Angst etwas zu verpassen, wenn du nicht unter Menschen bist?':'FOMO',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [spontan]':'Spontan',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [verwirrt]':'Verwirrt',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [glücklich]':'Glücklich',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [geduldig]':'Geduldig',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [streitlustig]':'Streitlustig',
    'Wie … schätzt Du dich selbst ein? (1 gar nicht - 5 sehr) [abenteuerlustig]':'Abenteuerlustig',
    'Hattest du schon einmal Zweifel daran, ob du dein Studium schaffen kannst?':'Studium_Zweifel',
    'Wie zufrieden bist du mit deinem Studium? ':'Studium_Zufriedenheit',
    'Strebst du nach dem Studium auch weiterhin eine akademische Laufbahn an?':'Akademische_Laufbahn',
    'Wie gern lernst Du in der Gruppe?':'Lerngruppe',
    'Prokrastinierst du, wenn du lernen müsstest?':'Prokrastination',
    'Wie häufig machst du Abgaben auf den letzten Drücker?':'Abgaben_spaet',
    'Wie gerne probierst du neue Lebensmittel aus?':'Neugier_Lebensmittel',
    'Wie häufig verbringst du Zeit in der Natur (z. B. beim Wandern oder Camping)?':'Natur',
    'Wie häufig verreist du?':'Verreisen',
    'Wie interessiert bist du an handwerklichen Tätigkeiten?':'Handwerk',
    'Wie gerne magst du Sprachnachrichten?':'Sprachnachrichten',
    'Wie wichtig ist es dir viel Geld zu verdienen?':'Geld'
}
data=data.rename(columns=to_change)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 48 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Timestamp                    62 non-null     object
 1   Alter                        62 non-null     int64 
 2   Geschlecht                   62 non-null     object
 3   Pseudonym                    62 non-null     object
 4   Position                     62 non-null     object
 5   Studiengang                  62 non-null     object
 6   Sportlichkeit                62 non-null     int64 
 7   Sportarten                   62 non-null     object
 8   Fav_Sportart                 62 non-null     object
 9   Gesunde_Ernaehrung           62 non-null     int64 
 10  Essen_gehen                  62 non-null     int64 
 11  Kochen                       62 non-null     int64 
 12  Feiern                       62 non-null     int64 
 13  Fav_Essen                    58 non-n

### NaN-Werte
Damit auch leere Einträge in die Darstellung mit einfließen können, werden diese als 'Keine Angabe' statt als NaN definiert.

In [143]:
data = data.fillna('Keine Angabe')

In [144]:
data.describe(include=['object'])

,Timestamp,Geschlecht,Pseudonym,Position,Studiengang,Sportarten,Fav_Sportart,Fav_Essen,Fav_Musik
count,62,62,62,62,62,62,62,62,62
unique,62,2,62,3,4,55,43,39,52
top,2023/05/25 9:05:15 AM GMT+3,männlich,lalala-lololo,Student*in,MMI,Bouldern/Klettern,Fußball,Pizza,Keine Angabe
freq,1,44,1,59,25,4,6,6,7


### Unnötige Leerzeichen und Sonderzeichen bereinigen

In [145]:
columns = ['Fav_Sportart', 'Fav_Essen', 'Fav_Musik']
emoji_unicodes = r'[\U0001F300-\U0001F5FF]'

for column in columns:
    data[column] = data[column].str.strip()
    emojis = re.findall(emoji_unicodes, str(data[column]))
    for e in emojis:
        data[column] = data[column].str.replace(e, '')

In [146]:
data['Fav_Sportart'].value_counts()

Fußball                             10
Badminton                            3
Bouldern                             3
Basketball                           3
Tischtennis                          3
Reiten                               2
Volleyball                           2
Klettern                             2
Tauchen                              2
/                                    1
Fahrrad fahren                       1
Tanzen                               1
Kraftsport und Kampfsport            1
Joggen                               1
Karate                               1
Hockey                               1
Squash                               1
Yoga                                 1
Inlineskaten                         1
Joggen/Laufen und Rennrad fahren     1
keine                                1
Ruderboot fahren                     1
Padel-Tennis                         1
Bogenschießen                        1
Radfahren                            1
E-Sport / CS:GO          

In [147]:
data['Fav_Essen'].value_counts()

Pizza                                       6
Burger                                      5
Keine Angabe                                4
Döner                                       4
Lasagne                                     4
Pfannkuchen                                 3
Pasta                                       3
Sushi                                       3
Nudeln                                      2
Hamburger                                   1
Tortelini                                   1
Yaprak                                      1
Lachs mit Bratkartoffeln und Gurkensalat    1
Ramen                                       1
Nudeln mit Pilzen in Sahnesauce             1
Humus                                       1
Pommes                                      1
Reispfanne mit viel Gemüse                  1
Gemüsepfanne mit Reis                       1
Carbonara                                   1
Ofenkartoffeln mit Gemüse                   1
Cordon bleu                       

In [148]:
data['Fav_Musik'].value_counts()

Keine Angabe                          7
Antilopen Gang                        2
Radiohead                             2
Mac Miller                            2
Drake                                 2
Kraftklub                             1
Alle                                  1
System of a Down                      1
Sting                                 1
Roger Cicero                          1
Keiner                                1
Avatar                                1
Måneskin                              1
Hans Zimmer                           1
Beethoven                             1
6lack                                 1
King Gizzard and the Lizard Wizard    1
NF                                    1
Deathspell Omega                      1
Red Hot Chili Peppers                 1
Grimes                                1
Die Toten Hosen                       1
Halsey                                1
Ed sheeran                            1
Neffex                                1


### Nicht-valide Einträge vereinheitlichen

In [149]:
invalid = [
    r'^(?<!\S)-(?!\S)$',    # alleinstehender Bindestrich
    r'Chillen',             # invalides Wort/Satz
    r'Weiss nicht',         # invalides Wort/Satz
    r'^(?<!\S)/(?!\S)$',    # alleinstehender Schrägstrich
    r'keine',               # invalides Wort/Satz
    r'Keiner',              # invalides Wort/Satz
    r'Alle'                 # invalides Wort/Satz
]
replacement = 'Keine Angabe'

In [150]:
for column in columns:
    for word in invalid:
        data[column] = data[column].str.replace(word, 'Keine Angabe', regex=True)

In [151]:
data['Fav_Sportart'].value_counts()

Fußball                             10
Keine Angabe                         5
Bouldern                             3
Basketball                           3
Tischtennis                          3
Badminton                            3
Reiten                               2
Tauchen                              2
Klettern                             2
Volleyball                           2
Fahrrad fahren                       1
Tanzen                               1
Kraftsport und Kampfsport            1
Joggen                               1
Bogenschießen                        1
Squash                               1
Yoga                                 1
Inlineskaten                         1
Joggen/Laufen und Rennrad fahren     1
Ruderboot fahren                     1
Padel-Tennis                         1
Hockey                               1
Radfahren                            1
Karate                               1
E-Sport / CS:GO                      1
Fitness                  

In [152]:
data['Fav_Essen'].value_counts()

Pizza                                       6
Burger                                      5
Keine Angabe                                4
Döner                                       4
Lasagne                                     4
Pfannkuchen                                 3
Pasta                                       3
Sushi                                       3
Nudeln                                      2
Hamburger                                   1
Tortelini                                   1
Yaprak                                      1
Lachs mit Bratkartoffeln und Gurkensalat    1
Ramen                                       1
Nudeln mit Pilzen in Sahnesauce             1
Humus                                       1
Pommes                                      1
Reispfanne mit viel Gemüse                  1
Gemüsepfanne mit Reis                       1
Carbonara                                   1
Ofenkartoffeln mit Gemüse                   1
Cordon bleu                       

In [153]:
data['Fav_Musik'].value_counts()

Keine Angabe                          10
Antilopen Gang                         2
Mac Miller                             2
Radiohead                              2
Drake                                  2
Kraftklub                              1
System of a Down                       1
Sting                                  1
Roger Cicero                           1
Avatar                                 1
Måneskin                               1
Hans Zimmer                            1
Beethoven                              1
6lack                                  1
NF                                     1
King Gizzard and the Lizard Wizard     1
Deathspell Omega                       1
Red Hot Chili Peppers                  1
Grimes                                 1
Die Toten Hosen                        1
Halsey                                 1
Ed sheeran                             1
Neffex                                 1
Haftbefehl                             1
Tyler the Creato

### Korrekturen, Abtrennung und Vereinheitlichung

In [154]:
to_cutoff = [r'\/.*', r'und.*']
for column in columns:
    for word in to_cutoff:
        data[column] = data[column].str.replace(word, '', regex=True)

# Fav_Sportart
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace('Ruderboot fahren', 'Rudern')
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace(r'^(?<!\S)Joggen(?!\S)$', 'Joggen/Laufen', regex=True)
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace(r'^(?<!\S)Klettern(?!\S)$', 'Bouldern/Klettern', regex=True)
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace(r'^(?<!\S)Bouldern(?!\S)$', 'Bouldern/Klettern', regex=True)
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace('Tisch-Tennis', 'Tischtennis')
data['Fav_Sportart'] = data['Fav_Sportart'].str.replace('Calesthenics','Calisthenics')

# Fav_Essen
data['Fav_Essen'] = data['Fav_Essen'].str.replace('Tortelini', 'Tortellini')

# Fav_Musik
data['Fav_Musik'] = data['Fav_Musik'].str.replace('Red hot Chili peppers', 'Red Hot Chili Peppers')

In [155]:
data['Fav_Sportart'].value_counts()

Fußball              10
Keine Angabe          5
Bouldern/Klettern     5
Tischtennis           4
Badminton             3
Basketball            3
Tauchen               2
Reiten                2
Volleyball            2
Joggen/Laufen         2
Calisthenics          2
Fahrrad fahren        1
Hockey                1
Squash                1
Yoga                  1
Bogenschießen         1
Inlineskaten          1
Rudern                1
Kraftsport            1
Padel-Tennis          1
Tanzen                1
Fitness               1
Karate                1
E-Sport               1
Radfahren             1
Schwimmen             1
Thaiboxen             1
Football              1
Kampfsport            1
Tennis                1
Schnorcheln           1
Trampolin             1
Ski fahren            1
Name: Fav_Sportart, dtype: int64

In [156]:
data['Fav_Essen'].value_counts()

Pizza                              6
Burger                             5
Keine Angabe                       4
Döner                              4
Lasagne                            4
Pfannkuchen                        3
Pasta                              3
Sushi                              3
Nudeln                             2
Hamburger                          1
Tortellini                         1
Yaprak                             1
Lachs mit Bratkartoffeln           1
Ramen                              1
Nudeln mit Pilzen in Sahnesauce    1
Humus                              1
Pommes                             1
Reispfanne mit viel Gemüse         1
Gemüsepfanne mit Reis              1
Carbonara                          1
Ofenkartoffeln mit Gemüse          1
Cordon bleu                        1
Vegane Lasagne                     1
Gurke mit Mozzarella               1
Shakshuka                          1
Burrito                            1
Butterbrot                         1
R

In [157]:
data['Fav_Musik'].value_counts()

Keine Angabe                          10
Antilopen Gang                         2
Red Hot Chili Peppers                  2
Radiohead                              2
Mac Miller                             2
Drake                                  2
Kraftklub                              1
System of a Down                       1
Sting                                  1
Roger Cicero                           1
Avatar                                 1
Måneskin                               1
Hans Zimmer                            1
Beethoven                              1
6lack                                  1
King Gizzard and the Lizard Wizard     1
NF                                     1
Deathspell Omega                       1
Grimes                                 1
Die Toten Hosen                        1
Halsey                                 1
Ed sheeran                             1
Neffex                                 1
Haftbefehl                             1
Tyler the Creato

## Umformatierungen

### String-Kategorien als numerische Daten ergänzen
- Geschlecht
    - 0 = weiblich
    - 1 = männlich
    - 2 = divers
- Position
    - 0 = Student\*in
    - 1 = Wissenschaftliche\*r Mitarbeiter\*in
    - 2 = Dozent\*in
- Studiengang
    - 0 = BMI
    - 1 = DAISY
    - 2 = BMT
    - 3 = MMI
- Sportarten
    - 0 = Basketball
    - 1 = Fußball
    - 2 = Handball
    - 3 = Football
    - 4 = Hockey
    - 5 = Volleyball
    - 6 = Tennis
    - 7 = Yoga
    - 8 = Kampfsport
    - 9 = Tanzen
    - 10 = Leichtathletik
    - 11 = Bouldern/Klettern
    - 12 = Schwimmen
    - 13 = Wassersport
    - 14 = Reiten
    - 15 = Wintersport
    - 16 = Rennrad/Mountainbike

In [158]:
data['Geschlecht_int'] = data['Geschlecht'].map({'weiblich':0, 'männlich':1, 'divers':2})
data['Position_int'] = data['Position'].map({'Student*in':0, 'Wissenschaftliche*r Mitarbeiter*in':1, 'Dozent*in':2})
data['Studiengang_int'] = data['Studiengang'].map({'BMI':0, 'DAISY':1, 'BMT':2, 'MMI':3})

In [159]:
sportarten_list = [
    'Basketball', 'Fußball', 'Handball', 
    'Football', 'Floorball', 'Hockey', 
    'Volleyball', 'Tennis', 'Yoga', 
    'Kampfsport', 'Tanzen', 'Leichtathletik', 
    'Bouldern/Klettern', 'Schwimmen', 'Wassersport', 
    'Reiten', 'Wintersport', 'Rennrad/Mountainbike'
    ]

for sportart in sportarten_list:
    # Wenn die Sportart im String vorkommt, eine 1 eintragen, andernfalls eine 0
    data[sportart] = data['Sportarten'].str.contains(sportart).astype(int)

# Rausfiltern von Sportarten die nicht ausgwählt wurden
all_zero_rows = (data[sportarten_list] == 0).all(axis=0)
columns_to_delete = all_zero_rows[all_zero_rows].index
data = data.drop(columns=columns_to_delete)

data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 68 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Timestamp                    62 non-null     object
 1   Alter                        62 non-null     int64 
 2   Geschlecht                   62 non-null     object
 3   Pseudonym                    62 non-null     object
 4   Position                     62 non-null     object
 5   Studiengang                  62 non-null     object
 6   Sportlichkeit                62 non-null     int64 
 7   Sportarten                   62 non-null     object
 8   Fav_Sportart                 62 non-null     object
 9   Gesunde_Ernaehrung           62 non-null     int64 
 10  Essen_gehen                  62 non-null     int64 
 11  Kochen                       62 non-null     int64 
 12  Feiern                       62 non-null     int64 
 13  Fav_Essen                    62 non-n

## Bereinigte Daten abspeichern

In [160]:
file_name = "./data/survery_data_annonymized_230626_clean.csv"
data.to_csv(file_name, sep=',')